In [1]:
!nvidia-smi

Sat Dec 21 16:23:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Install libraries

In [2]:
%%capture
!pip install git+https://github.com/huggingface/transformers.git
!pip install flax jsonlines ftfy wandb
!pip install -U tqdm
!pip install git+https://github.com/deepmind/optax.git

In [3]:
%%capture
!git clone https://github.com/arampacha/CLIP-rsicd

In [4]:
import sys
sys.path += ['CLIP-rsicd']

## Imports

In [5]:
import jax
try:
    # This will raise an exception if no TPU is found
    jax.devices('cuda')[0]
    half_precision_dtype = "bfloat16"
except IndexError:
    half_precision_dtype = "float16"
    print("No TPU detected, using float16")
device_count = jax.device_count()
print(device_count)

1


In [6]:
import logging
import math
import os
import sys
import time
from dataclasses import dataclass, field
from pathlib import Path
from typing import Callable, Optional, List
import json
import jsonlines
import shutil
import numpy as np
import torch

from flax import training
from tqdm.auto import tqdm

import jax
import jax.numpy as jnp
import optax
import transformers
from flax import jax_utils, traverse_util
from flax.jax_utils import unreplicate
from flax.training import train_state
from flax.training.common_utils import get_metrics, onehot, shard, shard_prng_key
from flax.training.checkpoints import save_checkpoint, restore_checkpoint
from flax.serialization import to_bytes, from_bytes
from transformers import (
    CONFIG_MAPPING,
    AutoConfig,
    FlaxCLIPModel,
    CLIPProcessor,
    CLIPTokenizer,
    TrainingArguments,
    is_tensorboard_available,
    IntervalStrategy

)
from transformers.testing_utils import CaptureLogger

from importlib.util import find_spec

logger = logging.getLogger(__name__)

In [7]:
from utils.data import ImageTextDataset, Transform

## Data

In this notebook we will be using [RSICD dataset](). A copy of this dataset is shared by the authors on google drive. To use it one can simply add a shrtcut to own gdrive. In my case it's `RSICD` folder.

In [8]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [9]:
from pathlib import Path
path = Path(root_dir + 'RSICD/')

In [10]:
%%capture
!unrar x gdrive/MyDrive/RSICD/RSICD_images.rar ./data/

In [11]:
!mv /content/data/RSICD_images/ /content/data/rsicd_images

mv: cannot stat '/content/data/RSICD_images/': No such file or directory


We'll be using augmented captions for the images. The files containing those are available in our [github repo]().

In [12]:
%%capture
!wget -P ./data/ https://raw.githubusercontent.com/arampacha/CLIP-rsicd/master/data/textaug_train_rsicd.jsonl
!wget -P ./data/ https://raw.githubusercontent.com/arampacha/CLIP-rsicd/master/data/textaug_valid_rsicd.jsonl
!wget -P ./data/ https://raw.githubusercontent.com/arampacha/CLIP-rsicd/master/data/textaug_test_rsicd.jsonl

Each line in the `jsonl` file has two fields: `filename` and `captions`:

In [13]:
import jsonlines
with jsonlines.open("/content/data/textaug_valid_rsicd.jsonl", "r") as reader:
    line = next(iter(reader))
print(f"filename: {line['filename']}\n")
print("captions:")
for i, caption in enumerate(line["captions"]):
    print(f"{i+1}) {caption}")

filename: rsicd_images/airport_61.jpg

captions:
1) the asphalted and airport runways divide the field into several rounded rectangles arranged next to which buildings and a road are located.
2) the tarmac and airport runways divide the field into several orderly arranged rounded rectangles  next to which is buildings and a road .
3) a brown ground divided by the grey runway .
4) we can see a simple termial building and an apron connceted with runways .
5) some building with a parking lot are near an airport with several runways .


## Training arguments

In [14]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir = "./tmp/model1",
    overwrite_output_dir = False,
    do_train = True,
    do_eval = True,
    evaluation_strategy = "steps",
    per_device_train_batch_size = 32,
    per_device_eval_batch_size = 32,
    gradient_accumulation_steps = 8,
    learning_rate = 5e-05,
    weight_decay = 0.1,
    adam_beta1 = 0.9,
    adam_beta2 = 0.98,
    adam_epsilon = 1e-06,
    max_grad_norm = 1.0,
    num_train_epochs = 1,
    max_steps = -1,
    lr_scheduler_type = "linear",
    warmup_ratio = 0.2,
    # warmup_steps = 0,
    logging_first_step = False,
    logging_steps = 50,
    save_strategy = "epoch",
    seed = 42,
    dataloader_drop_last = True,
    eval_steps = 200,
    run_name = None,
    adafactor = False,
    report_to = "all",
    skip_memory_metrics = True,
    resume_from_checkpoint = None,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1573: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [17]:
from dataclasses import dataclass, field
from typing import Optional
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune, or train from scratch.
    """

    model_name_or_path: Optional[str] = field(
        default=None,
        metadata={
            "help": "The model checkpoint for weights initialization."
            "Don't set if you want to train a model from scratch."
        },
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    dtype: Optional[str] = field(
        default="float32",
        metadata={
            "help": "Floating-point format in which the model weights should be initialized and trained. Choose one of `[float32, float16, bfloat16]`."
        },
    )
    save_optimizer: Optional[bool] = field(
        default=True,
        metadata={"help": "Whether to store full train state including optimizer."},
    )


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    data_dir: Optional[str] = field(
        default=None, metadata={"help": "Path to local folder containing data files."}
    )
    train_file: Optional[str] = field(
        default=None, metadata={"help": "The input training data file (a jsonlines file)."}
    )
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file (a jsonlines file)."},
    )
    train_file: Optional[str] = field(default=None, metadata={"help": "The input training data file (a text file)."})
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file to evaluate the perplexity on (a text file)."},
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    validation_split_percentage: Optional[int] = field(
        default=5,
        metadata={
            "help": "The percentage of the train set used as validation set in case there's no validation split"
        },
    )
    block_size: Optional[int] = field(
        default=None,
        metadata={
            "help": "Optional input sequence length after tokenization. "
            "The training dataset will be truncated in block of this size for training. "
            "Default to the model max input length for single sentence inputs (take into account special tokens)."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    text_column_name: Optional[str] = field(
            default='text',
            metadata={"help": "Column containing main text data."},
    )
    augment_images: Optional[bool] = field(
        default=True,
        metadata={ "help": "Augment input training images" }
    )
    augment_captions: Optional[bool] = field(
        default=True,
        metadata={ "help": "Augment input training images" }
    )
    captions_per_image: Optional[int] = field(
        default=5,
        metadata={"help": "Number of captions per image to use when creating train dataset."},
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in ["csv", "json", "txt", "jsonl"], "`train_file` should be a csv, a json or a txt file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in ["csv", "json", "txt", "jsonl"], "`validation_file` should be a csv, a json or a txt file."


@dataclass
class ImageAugmentationArguments:
    """
    Arguments for image augmentations configuration
    """
    random_horizontal_flip: Optional[float] = field(
        default=0.5,
        metadata={ "help": "Probability of applying random horizontal flip" }
    )
    random_vertical_flip: Optional[float] = field(
        default=0.5,
        metadata={ "help": "Probability of applying random vartical flip" }
    )


In [18]:
data_args = DataTrainingArguments(
    dataset_name = None,
    data_dir = "./data",
    train_file = "/content/data/textaug_train_rsicd.jsonl",
    validation_file = "/content/data/textaug_valid_rsicd.jsonl",
    max_train_samples = None,
    max_eval_samples = None,
    overwrite_cache = False,
    validation_split_percentage = 5,
    preprocessing_num_workers = 2,
    augment_images = True,
    augment_captions = True,
    captions_per_image = 5,
)

augmentation_args = ImageAugmentationArguments()

In [19]:
model_args = ModelArguments(
    model_name_or_path = "openai/clip-vit-base-patch32",
    dtype = half_precision_dtype,
    save_optimizer = False,
)

Some utility functions for training:

In [20]:
from flax.training import train_state
from typing import Callable
import jax.numpy as jnp
from flax import jax_utils
from flax.training.common_utils import shard_prng_key
class TrainState(train_state.TrainState):
    dropout_rng: jnp.ndarray

    def replicate(self):
        return jax_utils.replicate(self).replace(dropout_rng=shard_prng_key(self.dropout_rng))


def write_train_metric(summary_writer, train_metrics, train_time, step):
    summary_writer.scalar("train_time", train_time, step)

    train_metrics = get_metrics(train_metrics)
    for key, vals in train_metrics.items():
        tag = f"train_{key}"
        for i, val in enumerate(vals):
            summary_writer.scalar(tag, val, step - len(vals) + i + 1)


def write_eval_metric(summary_writer, eval_metrics, step):
    for metric_name, value in eval_metrics.items():
        summary_writer.scalar(f"eval_{metric_name}", value, step)


def create_learning_rate_fn(
    train_ds_size: int, train_batch_size: int, num_train_epochs: int, warmup_ratio: float, learning_rate: float
) -> Callable[[int], jnp.array]:
    """Returns a linear warmup, linear_decay learning rate function."""
    steps_per_epoch = train_ds_size // train_batch_size
    num_train_steps = steps_per_epoch * num_train_epochs
    num_warmup_steps = int(num_train_steps * warmup_ratio)
    warmup_fn = optax.linear_schedule(init_value=0.0, end_value=learning_rate, transition_steps=num_warmup_steps)
    decay_fn = optax.linear_schedule(
        init_value=learning_rate, end_value=0, transition_steps=num_train_steps - num_warmup_steps
    )
    schedule_fn = optax.join_schedules(schedules=[warmup_fn, decay_fn], boundaries=[num_warmup_steps])
    return schedule_fn

# utils
def mb_item(x):
    return x.item() if hasattr(x, "item") else x

def make_batch(samples):
    batch = {k:jnp.array(v) for k,v in samples.items()}
    return batch


## Training

In [21]:
# Make one log on every process with the configuration for debugging.
import logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)

In [22]:
# Load pretrained model and tokenizer
from transformers import AutoConfig, CLIPProcessor, FlaxCLIPModel, CONFIG_MAPPING, is_tensorboard_available
# Distributed training:
# The .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.
if model_args.config_name:
    config = AutoConfig.from_pretrained(model_args.config_name, cache_dir=model_args.cache_dir)
elif model_args.model_name_or_path:
    config = AutoConfig.from_pretrained(model_args.model_name_or_path, cache_dir=model_args.cache_dir)
else:
    config = CONFIG_MAPPING[model_args.model_type]()
    logger.warning("You are instantiating a new config instance from scratch.")

processor = CLIPProcessor.from_pretrained(model_args.model_name_or_path)
tokenizer = processor.tokenizer
if model_args.model_name_or_path:
    model = FlaxCLIPModel.from_pretrained(
        model_args.model_name_or_path, config=config, seed=training_args.seed, dtype=getattr(jnp, model_args.dtype)
    )
else:
    model = FlaxCLIPModel.from_config(
        config, seed=training_args.seed, dtype=getattr(jnp, model_args.dtype)
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

flax_model.msgpack:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [31]:
from torch.utils.data import Dataset
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image = self.df.iloc[idx]['image']
        caption = self.df.iloc[idx]['caption']

        if self.transform:
            image = self.transform(image)  # Apply your image transformations

        # Assuming your `transform` returns a PIL Image, convert to NumPy array:
        #image = np.array(image)

        return image, caption

In [25]:
from PIL import Image
import io
import ast
import pandas as pd
def convert_bytes_to_Image(byte_array) :
    return Image.open(io.BytesIO((ast.literal_eval(byte_array)['bytes'])))
df = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/rsicd/train.csv')
def modify_sentences(sentence):
    # Split the multi-line string into individual sentences
    #sentences = sentence.split('\n')

    # Parse the string representation of the list into a list of strings
    captions_list = [sentence.strip() for sentence in sentence.strip("[]").split('\n')]

    #array_of_strings = [sentence.strip() for sentence in parsed_list]
    return captions_list
df['captions'] = df['captions'].apply(lambda x: modify_sentences(x))
df['captions'].tolist()
print(df['captions'][0][0][1:-1])
df['caption']=[i[0][1:-1] for i in df['captions']] #use only one caption for now
df['image'] = df['image'].apply(lambda x: convert_bytes_to_Image(x))
df['filename'] = df['filename'].str[13:]
dfvalid = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/rsicd/valid.csv')
dfvalid['captions'] = dfvalid['captions'].apply(lambda x: modify_sentences(x))
dfvalid['captions'].tolist()
print(dfvalid['captions'][0][0][1:-1])
dfvalid['caption']=[i[0][1:-1] for i in dfvalid['captions']] #use only one caption for now
#convert_bytes_to_Image(df['image'][1])
dfvalid['image'] = dfvalid['image'].apply(lambda x: convert_bytes_to_Image(x))
dfvalid['filename'] = dfvalid['filename'].str[13:]

Many aircraft are parked next to a long building in an airport.
the asphalted and airport runways divide the field into several rounded rectangles arranged next to which buildings and a road are located.


In [35]:
from torchvision import transforms
class ToTensorModule(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return transforms.to_tensor(x)

In [36]:
import torch
from utils.data import ImageTextDataset, Transform
from torchvision import transforms
config = model.config
# Initialize torchvision transforms and jit them for faster processing
preprocess = Transform(config.vision_config.image_size, data_args.augment_images, augmentation_args)
#preprocess = torch.jit.script(preprocess)
# Change 1: Before jit scripting convert Transform to a torch.nn.Module
preprocess = torch.jit.script(torch.nn.Sequential(ToTensorModule(), preprocess))

eval_preprocess = Transform(config.vision_config.image_size, False, augmentation_args)
#eval_preprocess = torch.jit.script(eval_preprocess)
# Change 2: Before jit scripting convert Transform to a torch.nn.Module
eval_preprocess = torch.jit.script(torch.nn.Sequential(ToTensorModule(), eval_preprocess))

# Initialize the image-text dataset
'''train_dataset = ImageTextDataset(
    data_args.data_dir,
    "train",
    captions_per_image=data_args.captions_per_image,
    augment_captions=data_args.augment_captions,
    transform=preprocess,
)

eval_dataset = ImageTextDataset(
    data_args.data_dir,
    "valid",
    captions_per_image=1,
    transform=eval_preprocess,
)'''
train_dataset = CustomDataset(df=df, transform=preprocess)
eval_dataset = CustomDataset(df=dfvalid, transform=eval_preprocess)


# Enable tensorboard only on the master node
has_tensorboard = is_tensorboard_available()
if has_tensorboard and jax.process_index() == 0:
    try:
        from flax.metrics.tensorboard import SummaryWriter

        summary_writer = SummaryWriter(log_dir=Path(training_args.output_dir))
    except ImportError as ie:
        has_tensorboard = False
        logger.warning(
            f"Unable to display metrics through TensorBoard because some package are not installed: {ie}"
        )
else:
    logger.warning(
        "Unable to display metrics through TensorBoard because the package is not installed: "
        "Please run pip install tensorboard to enable."
    )

# Use collate function to tokenizer the text and convert the processed images to numpy
def collate_fn(examples):
    pixel_values = torch.stack([example[0] for example in examples]).numpy()
    captions = [example[1] for example in examples]
    inputs = tokenizer(captions, max_length=32, padding="max_length", return_tensors="np", truncation=True)

    batch = {
        "pixel_values": pixel_values,
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
    }

    return batch

# Store some constant
num_epochs = int(training_args.num_train_epochs)
train_batch_size = int(training_args.per_device_train_batch_size) * jax.device_count() * training_args.gradient_accumulation_steps
eval_batch_size = int(training_args.per_device_eval_batch_size) * jax.device_count()
steps_per_epoch = len(train_dataset) // train_batch_size
total_train_steps = steps_per_epoch * num_epochs

# Create data loaders
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=train_batch_size,
    shuffle=True,
    num_workers=data_args.preprocessing_num_workers,
    persistent_workers=True,
    drop_last=True,
    collate_fn=collate_fn,
)

eval_loader = torch.utils.data.DataLoader(
    eval_dataset,
    batch_size=eval_batch_size,
    shuffle=False,
    num_workers=data_args.preprocessing_num_workers,
    persistent_workers=True,
    drop_last=True,
    collate_fn=collate_fn,
)


RuntimeError: 
Module 'ToTensorModule' has no attribute 'to_tensor' (This attribute exists on the Python module, but we failed to convert Python type: 'torchvision.transforms.transforms.ToTensor' to a TorchScript type. Only tensors and (possibly nested) tuples of tensors, lists, or dictsare supported as inputs or outputs of traced functions, but instead got value of type ToTensor.. Its type was inferred; try adding a type annotation for the attribute.):
  File "<ipython-input-35-aea48ee7464d>", line 7
    def forward(self, x):
        return self.to_tensor(x)
               ~~~~~~~~~~~~~~ <--- HERE


In [27]:
# enable wandb tracking
from importlib.util import find_spec # import find_spec from importlib.util
import optax
from flax import traverse_util

has_wandb = find_spec("wandb") is not None
if jax.process_index() == 0 and has_wandb and ("wandb" in training_args.report_to):
    try:
        import wandb
        if training_args.run_name is None:
            run_name = training_args.output_dir.split("/")[-1]
        else:
            run_name = training_args.run_name
        wandb.init(
            name=run_name,
            entity="wandb",
            project="hf-flax-clip-rsicd",
            sync_tensorboard=True
        )
        wandb.config.update(training_args)
        wandb.config.update(model_args)
        wandb.config.update(data_args)
    except ImportError as e:
        print(e)
        has_wandb = False

# Initialize our training
rng = jax.random.PRNGKey(training_args.seed)
rng, dropout_rng = jax.random.split(rng)

# Create learning rate schedule
linear_decay_lr_schedule_fn = create_learning_rate_fn(
    len(train_dataset),
    train_batch_size,
    training_args.num_train_epochs,
    training_args.warmup_ratio,
    training_args.learning_rate,
)

# We use Optax's "masking" functionality to not apply weight decay
# to bias and LayerNorm scale parameters. decay_mask_fn returns a
# mask boolean with the same structure as the parameters.
# The mask is True for parameters that should be decayed.
# Note that this mask is specifically adapted for FlaxGPT2.
# For other models, one should correct the layer norm parameter naming
# accordingly.
def decay_mask_fn(params):
    flat_params = traverse_util.flatten_dict(params)
    flat_mask = {
        path: (path[-1] != "bias" and path[-2:] not in [("ln_1", "scale"), ("ln_2", "scale"), ("ln_f", "scale")])
        for path in flat_params
    }
    return traverse_util.unflatten_dict(flat_mask)

# create optimizer
if training_args.adafactor:
    # We use the default parameters here to initialize adafactor,
    # For more details about the parameters please check https://github.com/deepmind/optax/blob/ed02befef9bf81cbbf236be3d2b0e032e9ed4a40/optax/_src/alias.py#L74
    optimizer = optax.adafactor(
        learning_rate=linear_decay_lr_schedule_fn,
    )
else:
    optimizer = optax.adamw(
        learning_rate=linear_decay_lr_schedule_fn,
        b1=training_args.adam_beta1,
        b2=training_args.adam_beta2,
        eps=training_args.adam_epsilon,
        weight_decay=training_args.weight_decay,
        mask=decay_mask_fn,
    )
    optimizer = optax.chain(
        optax.clip_by_global_norm(training_args.max_grad_norm),
        optimizer
    )
if training_args.gradient_accumulation_steps > 1:
    optimizer = optax.MultiSteps(optimizer, training_args.gradient_accumulation_steps)
grad_accum_steps = training_args.gradient_accumulation_steps

# Setup train state
state = TrainState.create(apply_fn=model.__call__, params=model.params, tx=optimizer, dropout_rng=dropout_rng)

if training_args.resume_from_checkpoint:
    state = restore_checkpoint(training_args.resume_from_checkpoint, state)
    resume_step = mb_item(state.step)
else:
    resume_step = 0

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [28]:
def cross_entropy(logits, axis):
    logprobs = jax.nn.log_softmax(logits, axis=axis)
    nll = jnp.diag(logprobs)
    ce = -jnp.mean(nll)
    return ce

def clip_loss(similarity):
    loss = (cross_entropy(similarity, axis=0) + cross_entropy(similarity, axis=1)) / 2
    return loss

# Define gradient update step fn
def train_step(state, batch):
    dropout_rng, new_dropout_rng = jax.random.split(state.dropout_rng)

    def compute_loss(params):
        logits = state.apply_fn(**batch, params=params, dropout_rng=dropout_rng, train=True)[0]
        loss = clip_loss(logits)
        return loss

    grad_fn = jax.value_and_grad(compute_loss)
    loss, grad = grad_fn(state.params)
    grad = jax.lax.pmean(grad, "batch")

    new_state = state.apply_gradients(grads=grad, dropout_rng=new_dropout_rng)

    metrics = {"loss": loss, "learning_rate": linear_decay_lr_schedule_fn(state.step // grad_accum_steps)}
    metrics = jax.lax.pmean(metrics, axis_name="batch")

    return new_state, metrics

# Define eval fn
def eval_step(params, batch):
    logits = model(**batch, params=params, train=False)[0]
    loss = clip_loss(logits)

    # summarize metrics
    metrics = {"loss": loss}
    metrics = jax.lax.pmean(metrics, axis_name="batch")
    return metrics

# Create parallel version of the train and eval step
p_train_step = jax.pmap(train_step, "batch", donate_argnums=(0,))
p_eval_step = jax.pmap(eval_step, "batch")

# Replicate the train state on each device
state = state.replicate()

In [29]:
import logging
from tqdm import tqdm
import time

logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

logger.info("***** Running training *****")
logger.info(f"  Num examples = {len(train_dataset)}")
logger.info(f"  Num Epochs = {num_epochs}")
logger.info(f"  Instantaneous batch size per device = {training_args.per_device_train_batch_size}")
logger.info(f"  Total train batch size (w. parallel, distributed and grad_accum) = {train_batch_size}")
logger.info(f"  Total optimization steps = {total_train_steps}")

if not training_args.skip_memory_metrics:
    server = jax.profiler.start_server(9999)

train_time = 0
train_metrics = []
epochs = tqdm(range(num_epochs), desc=f"Epoch ... (1/{num_epochs})", position=0)
for epoch in epochs:
    # ======================== Training ================================
    train_start = time.time()

    # Create sampling rng
    rng, input_rng = jax.random.split(rng)

    # Generate an epoch by shuffling sampling indices from the train dataset
    steps_per_epoch = len(train_dataset) // train_batch_size
    # train
    steps_trained_progress_bar = tqdm(range(steps_per_epoch), desc="Training...", position=1,
                                        leave=False, initial=(resume_step // grad_accum_steps))
    for step, batch in enumerate(train_loader):
        cur_step = epoch * (len(train_dataset) // train_batch_size) + step
        # skip to the step from which we are resuming
        if cur_step < resume_step:
            continue

        batch = shard(make_batch(batch))

        state, train_metric = p_train_step(state, batch)
        train_metrics.append(train_metric)
        if step % grad_accum_steps == 0:
            steps_trained_progress_bar.update(1)

        if cur_step % (training_args.logging_steps * grad_accum_steps)== 0 and cur_step > 0:
            # Save metrics
            train_metric = unreplicate(train_metric)
            train_time += time.time() - train_start
            if has_tensorboard and jax.process_index() == 0:
                write_train_metric(summary_writer, train_metrics, train_time, cur_step)
            if has_wandb and jax.process_index() == 0 and ("wandb" in training_args.report_to):
                # TODO: add accumulation of metrics
                _metrics = {k if k=="learning_rate" else f"train_{k}":mb_item(v.mean()) for k, v in train_metric.items()}
                wandb.log({"training_step":cur_step, **_metrics}, commit=True)

            epochs.write(
                f"Step... ({cur_step} | Loss: {train_metric['loss'].mean()}, Learning Rate: {train_metric['learning_rate'].mean()})"
            )

            train_metrics = []

    if True:
        # ======================== Evaluating ==============================
        eval_metrics = []
        eval_steps = len(eval_dataset) // eval_batch_size
        for batch in tqdm(eval_loader, desc="Evaluating...", position=2, leave=False):
            # Model forward
            batch = shard(make_batch(batch))
            metrics = p_eval_step(state.params, batch)
            eval_metrics.append(metrics)

        # normalize eval metrics
        eval_metrics = get_metrics(eval_metrics)
        eval_metrics = jax.tree_map(jnp.mean, eval_metrics)

        # Print metrics and update progress bar
        desc = f"Step... ({cur_step} | Eval Loss: {eval_metrics['loss']})"
        epochs.write(desc)
        epochs.desc = desc

        # Save metrics
        if has_tensorboard and jax.process_index() == 0:
            # cur_step = epoch * (len(train_dataset) // train_batch_size)
            write_eval_metric(summary_writer, eval_metrics, cur_step)
        if has_wandb and jax.process_index() == 0 and ("wandb" in training_args.report_to):
            _metrics = {f"eval_{k}":mb_item(v) for k, v in eval_metrics.items()}
            wandb.log({"eval_step":cur_step, **_metrics})

    # save checkpoint after each epoch
    if jax.process_index() == 0:
        save_dir = f"{training_args.output_dir}/ckpt-{epoch}"
        model.save_pretrained(
            save_dir,
            params=unreplicate(state.params),
            push_to_hub=False, # training_args.push_to_hub, # we don't push intermediate steps
            commit_message=f"Saving weights and logs at epoch {epoch}",
            repo_name_or_path=training_args.output_dir
        )
        if model_args.save_optimizer:
            save_checkpoint(training_args.output_dir, unreplicate(state), cur_step, keep=training_args.save_total_limit, overwrite=True)
        if training_args.save_total_limit is not None:
            rotate_checkpoints(training_args.output_dir, training_args.save_total_limit)


Training...:   0%|          | 0/34 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch ... (1/1):   0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-23-89888ef6c992>", line 17, in __getitem__
    image = self.transform(image)  # Apply your image transformations
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 174